In [ ]:
import os
import pandas as pd
import urllib
from functools import partial
import multiprocessing
from tqdm import tqdm
from pathlib import Path
from time import time

# Ensure this path points to where you want to save the images
savefolder = r'C:/Users/sanja/Desktop/Amazon/cleanData/img'

# Load the dataset containing image links
df = pd.read_csv('C:/Users/sanja/Desktop/Amazon/data/train.csv')  # Adjust the path to your CSV file
image_links = df['image_link'].tolist()  # Get the list of image URLs

# Function to download each image
def download_image(image_link, savefolder, success_counter):
    if isinstance(image_link, str):
        filename = Path(image_link).name
        image_save_path = os.path.join(savefolder, filename)
        if not os.path.exists(image_save_path):
            try:
                urllib.request.urlretrieve(image_link, image_save_path)  
                success_counter[0] += 1  # Increment the counter for successful downloads
            except Exception as ex:
                print(f'Warning: Not able to download - {image_link}\n{ex}')
        else:
            success_counter[0] += 1  # Increment counter if image already exists
    return

# Function to download all images with progress tracking
def download_images(image_links, download_folder):
    total_images = len(image_links)
    print(f"Total images to download: {total_images}")

    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    # Initialize a success counter (used to track how many images have been successfully downloaded)
    success_counter = [0]

    # Use partial function to pass the success_counter along with the download_image function
    download_image_partial = partial(download_image, savefolder=download_folder, success_counter=success_counter)
    
    start_time = time()  # Start the timer to track total time
    
    with multiprocessing.Pool(20) as pool:
        for _ in tqdm(pool.imap(download_image_partial, image_links), total=total_images):
            pass  # No need to store results, just download images
    
    end_time = time()  # End the timer after downloading
    
    # Calculate and print download progress
    downloaded_images = success_counter[0]
    download_percentage = (downloaded_images / total_images) * 100
    print(f"\nDownloaded {downloaded_images} out of {total_images} images.")
    print(f"Download progress: {download_percentage:.2f}%")
    
    # Calculate and print the total time taken to download the images
    total_time = end_time - start_time
    print(f"Total time to download images: {total_time:.2f} seconds")

# Start downloading the images
download_images(image_links, savefolder)


Total images to download: 75000


  0%|                                                                                        | 0/75000 [00:00<?, ?it/s]